# 10 - Homework

In this homework, we'll deploy Credit Card prediction model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.

## Building the image

Clone the course repo if you haven't:

```
git clone https://github.com/alexeygrigorev/mlbookcamp-code.git
```

Go to the `course-zoomcamp/cohorts/2022/05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:v001 .
```

> **Note:** If you have troubles building the image, you can 
> use the image we built and published to docker hub:
> `svizor42/zoomcamp-model:v001`

```bash
docker build -t zoomcamp-model:v001 .
[+] Building 24.7s (11/11) FINISHED
 => [internal] load build definition from Dockerfile                                                          0.0s
 => => transferring dockerfile: 314B                                                                          0.0s
 => [internal] load .dockerignore                                                                             0.0s
 => => transferring context: 2B                                                                               0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.9.12-slim                                  2.8s
 => [auth] svizor/zoomcamp-model:pull token for registry-1.docker.io                                          0.0s
 => [internal] load build context                                                                             0.0s
 => => transferring context: 17.02kB                                                                          0.0s
 => [1/5] FROM docker.io/svizor/zoomcamp-model:3.9.12-slim@sha256:10445b40653d5ac17ede84db17f42ae8c4090b347a  0.0s
 => => resolve docker.io/svizor/zoomcamp-model:3.9.12-slim@sha256:10445b40653d5ac17ede84db17f42ae8c4090b347a  0.0s
 => CACHED [2/5] RUN pip install pipenv                                                                       0.0s
 => [3/5] COPY [Pipfile, Pipfile.lock, ./]                                                                    0.1s
 => [4/5] RUN pipenv install --system --deploy                                                               19.7s
 => [5/5] COPY [q6_predict.py, ./]                                                                            0.1s
 => exporting to image                                                                                        1.8s
 => => exporting layers                                                                                       1.7s
 => => writing image sha256:0b986ac8a1d3a5c653a98be10e388076e3df93c4084d74412edf187cbc8bd8e8                  0.0s
 => => naming to docker.io/library/zoomcamp-model:v001                                                        0.0s

Use 'docker scan' to run Snyk tests against images to find vulnerabilities and learn how to fix them
```

## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:v001
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```
{'get_card': True, 'get_card_probability': <value>}
```

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

* 0.289
* 0.502
* **0.769**
* 0.972

Now you can stop the container running in Docker.

## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

```bash
kind --version
kind version 0.17.0
```

## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```

```bash
kubectl cluster-info --context kind-kind
Kubernetes control plane is running at https://127.0.0.1:44153
CoreDNS is running at https://127.0.0.1:44153/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
```

## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

![kubernetes](kubernetes.png)

* Node
* **Pod**
* Deployment
* Service

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* **ClusterIP**
* NodePort
* LoadBalancer
* ExternalName

## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:v001`),
we need to register it with `kind`.

```bash
kind load docker-image zoomcamp-model:v001
Image: "" with ID "sha256:0b986ac8a1d3a5c653a98be10e388076e3df93c4084d74412edf187cbc8bd8e8" not yet present on node "kind-control-plane", loading...
```

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* **`kind load docker-image`**
* `kubectl apply`

## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit-card
spec:
  selector:
    matchLabels:
      app: credit-card
  replicas: 1
  template:
    metadata:
      labels:
        app: credit-card
    spec:
      containers:
      - name: credit-card
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`? **9696**

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

```bash
kubectl apply -f deployment.yaml
deployment.apps/credit-card created
```

```bash
kubectl get pod
NAME                           READY   STATUS    RESTARTS   AGE
credit-card-5ffc9786d4-qd76j   1/1     Running   0          11s
```

## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`? **credit-card**

Apply this config file.

```bash
kubectl apply -f service.yaml
service/credit-card created
```

## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

```bash
python q6_test.py
{'get_card': True, 'get_card_probability': 0.7692649226628628}
```

## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
horizontalpodautoscaler.autoscaling/credit-card-hpa autoscaled
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
NAME              REFERENCE                TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   <unknown>/20%   1         3         1          25s
```

The output should be similar to the next:

```bash
NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the credit-card service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.

```bash
kubectl get hpa
NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   20%/20%   1         3         2          32m
```

## Question 8 (optional)

Run `kubectl get hpa credit-card-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1
* **2**
* 3
* 4

>Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled 
> in any way it may happen that the final number of replicas will differ from initial.